#### ----------------------------------------------------------
## 🧩 Pandera - Schema Validation
#### ----------------------------------------------------------

- Pandera validates dataframes to ensure they follow the correct structure.
- It's great for runtime checks after cleaning or transformation steps.

In [3]:
##### REQUIREMENT: uv add pandera pandas

import pandas as pd
import pandera as pa
from pandera import Column, Check
from pandera.typing import DataFrame, Series

# ----------------------------------------------------------
# Load sample data
# ----------------------------------------------------------
df = pd.read_csv("/Users/riadanas/Desktop/ML-Engineering-Bootcamp/Artists.csv")
df.head()


,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,male,1936,0,NaN,NaN
2,3,Bill Arnold,"American, born 1941",American,male,1941,0,NaN,NaN
3,4,Charles Arnoldi,"American, born 1946",American,male,1946,0,Q1063584,500027998.0
4,5,Per Arnoldi,"Danish, born 1941",Danish,male,1941,0,NaN,NaN


In [4]:
# ----------------------------------------------------------
# Define schema for validation
# ----------------------------------------------------------
class ArtistSchema(pa.DataFrameModel):
    ConstituentID: Series[int] = pa.Field(gt=0)
    DisplayName: Series[str] = pa.Field(str_length={"min_value": 1, "max_value": 100})
    Nationality: Series[str] | None
    Gender: Series[str] = pa.Field(isin=["male", "female", "unknown"])
    BeginDate: Series[int] = pa.Field(in_range={"min_value": 1800, "max_value": 2025})
    EndDate: Series[int] | None

/Users/riadanas/Desktop/ML-Engineering-Bootcamp/.venv/lib/python3.13/site-packages/pandera/_pandas_deprecated.py:160: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [6]:
# ----------------------------------------------------------
# Validate dataframe
# ----------------------------------------------------------
try:
    validated_df: DataFrame[ArtistSchema] = ArtistSchema.validate(df)
    print("✅ All columns passed validation!")
except pa.errors.SchemaError as e:
    print("❌ Validation failed:")
    print(e)

❌ Validation failed:
Column 'DisplayName' failed element-wise validator number 0: str_length(1, 100) failure cases: SHE Kantha (Anima Sarkar, Tapati Roy, Kakoli Mandal, Sabina Bibi, Ajija Khatoon, Manjila Bibi Mondal)


In [5]:
# ----------------------------------------------------------
# Validate dataframe
# ----------------------------------------------------------
validated_df: DataFrame[ArtistSchema] = ArtistSchema.validate(df)
validated_df.head()

SchemaError: Column 'DisplayName' failed element-wise validator number 0: str_length(1, 100) failure cases: SHE Kantha (Anima Sarkar, Tapati Roy, Kakoli Mandal, Sabina Bibi, Ajija Khatoon, Manjila Bibi Mondal)

In [ ]:
# ----------------------------------------------------------
# ✅ Wrap-up
# ----------------------------------------------------------
# Pandera checks each column according to the schema above.
# If a column has wrong types or values, it raises a clear, readable error.
# This makes it perfect for validating intermediate DataFrames in a pipeline.